In [77]:
import os
import json
import requests
from gensim import corpora, models, similarities
from time import time
from nltk.tokenize import sent_tokenize,word_tokenize,regexp_tokenize
import pandas as pd
import operator
import numpy as np

In [2]:
json_data = open('squad_train_doc.json')
data = json.load(json_data)

In [6]:
data_train = pd.read_json('squad_train_doc.json')

In [7]:
for i 
data_train

,passages,title
0,[{'questions': ['To whom did the Virgin Mary a...,University_of_Notre_Dame
1,[{'questions': ['In what city did Beyonce grow...,Beyoncé
2,[{'questions': ['How many ranges are part of t...,Montana
3,[{'questions': ['What 2001 case was declared g...,Genocide
4,[{'questions': ['What does emergence of resist...,Antibiotics
5,[{'questions': ['In what area was Frédéric bor...,Frédéric_Chopin
6,[{'questions': ['Who were Wang Jiawei and Nyim...,Sino-Tibetan_relations_during_the_Ming_dynasty
7,[{'questions': ['What kind of device is the iP...,IPod
8,[{'questions': ['When was Twilight Princess la...,The_Legend_of_Zelda:_Twilight_Princess
9,[{'questions': ['How many James Bond films has...,Spectre_(2015_film)


In [9]:
data_train.iloc[0]

passages    [{'questions': ['To whom did the Virgin Mary a...
title                                University_of_Notre_Dame
Name: 0, dtype: object

In [13]:
docs_length = data_train.shape[0]

In [26]:
(data_train.iloc[0,0][0]['context']).split()

['Architecturally,',
 'the',
 'school',
 'has',
 'a',
 'Catholic',
 'character.',
 'Atop',
 'the',
 'Main',
 "Building's",
 'gold',
 'dome',
 'is',
 'a',
 'golden',
 'statue',
 'of',
 'the',
 'Virgin',
 'Mary.',
 'Immediately',
 'in',
 'front',
 'of',
 'the',
 'Main',
 'Building',
 'and',
 'facing',
 'it,',
 'is',
 'a',
 'copper',
 'statue',
 'of',
 'Christ',
 'with',
 'arms',
 'upraised',
 'with',
 'the',
 'legend',
 '"Venite',
 'Ad',
 'Me',
 'Omnes".',
 'Next',
 'to',
 'the',
 'Main',
 'Building',
 'is',
 'the',
 'Basilica',
 'of',
 'the',
 'Sacred',
 'Heart.',
 'Immediately',
 'behind',
 'the',
 'basilica',
 'is',
 'the',
 'Grotto,',
 'a',
 'Marian',
 'place',
 'of',
 'prayer',
 'and',
 'reflection.',
 'It',
 'is',
 'a',
 'replica',
 'of',
 'the',
 'grotto',
 'at',
 'Lourdes,',
 'France',
 'where',
 'the',
 'Virgin',
 'Mary',
 'reputedly',
 'appeared',
 'to',
 'Saint',
 'Bernadette',
 'Soubirous',
 'in',
 '1858.',
 'At',
 'the',
 'end',
 'of',
 'the',
 'main',
 'drive',
 '(and',
 'i

In [28]:
len(data_train.iloc[1,0])

66

In [37]:
docs_list = []
for i in range(docs_length):
    words = []
    for j in range(len(data_train.iloc[i,0])):
        words += ((data_train.iloc[i,0][j]['context']).split())
    docs_list.append(words)    
    

In [52]:
len(docs_list[441])

7054

In [81]:
sum = 0
for d in range(len(docs_list)):
    for k in range(len(docs_list[d])):
        sum += k
        
sum    

7073512290

In [47]:
def training(doc_list):
    dictionary = corpora.Dictionary(doc_list)
    dictionary.save('/tmp/harry.dict') # store the dictionary, for future reference
    #print(len(dictionary))
    raw_corpus = [dictionary.doc2bow(t) for t in doc_list]
    #print("Then convert convert tokenized documents to vectors: %s"% type(raw_corpus))
    corpora.MmCorpus.serialize('/tmp/harry.mm', raw_corpus) # store to disk
    #print("Save the vectorized corpus as a .mm file")
    dictionary = corpora.Dictionary.load('/tmp/harry.dict')
    corpus = corpora.MmCorpus('/tmp/harry.mm')
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    index = similarities.MatrixSimilarity(tfidf[corpus])
    index.save('/tmp/harry.index')
    index = similarities.MatrixSimilarity.load('/tmp/harry.index')
    sims = index[corpus_tfidf]
    #print("We get a similarity matrix for all documents in the corpus %s"% type(sims))
    #print(sims)
    return corpus_tfidf,index,tfidf

In [48]:
corpus_tfidf,index,tfidf = training(docs_list)

184853
Then convert convert tokenized documents to vectors: <class 'list'>
Save the vectorized corpus as a .mm file
We get a similarity matrix for all documents in the corpus <class 'numpy.ndarray'>
[[ 0.99999911  0.01463419  0.01416114 ...,  0.00628813  0.01065559
   0.01064884]
 [ 0.01463419  0.99999994  0.00946885 ...,  0.00276871  0.00439652
   0.00429137]
 [ 0.01416114  0.00946885  1.00000143 ...,  0.00526739  0.04814124
   0.00881663]
 ..., 
 [ 0.00628813  0.00276871  0.00526739 ...,  0.99999946  0.0148455
   0.00492789]
 [ 0.01065559  0.00439652  0.04814124 ...,  0.0148455   0.99999905
   0.00771047]
 [ 0.01064884  0.00429137  0.00881663 ...,  0.00492789  0.00771047
   0.99999982]]


In [55]:
def query_of_similarity1(t,index,tfidf):
    text = (word_tokenize(t))
    dictionary2 = corpora.Dictionary([text])
    dictionary2.save('/tmp/query.dict') 
    raw_corpus2 = [dictionary2.doc2bow(t) for t in [text]]
    corpora.MmCorpus.serialize('/tmp/rawcorpus2.mm', raw_corpus2) 
    dictionary2 = corpora.Dictionary.load('/tmp/query.dict')
    corpus2 = corpora.MmCorpus('/tmp/rawcorpus2.mm')
    r = index.get_similarities(tfidf[corpus2])
    return r

    

In [56]:
doc_selection =  query_of_similarity1("""As at most other universities, Notre Dame's students run a number of news media outlets. The nine student-run outlets include three newspapers, both a radio and television station, and several magazines and journals. Begun as a one-page journal in September 1876, the Scholastic magazine is issued twice monthly and claims to be the oldest continuous collegiate publication in the United States. The other magazine, The Juggler, is released twice a year and focuses on student literature and artwork. The Dome yearbook is published annually. The newspapers have varying publication interests, with The Observer published daily and mainly reporting university and other news, and staffed by students from both Notre Dame and Saint Mary's College. Unlike Scholastic and The Dome.""")

In [70]:

doc_pick, value = max(enumerate(doc_selection), key=operator.itemgetter(1))

In [71]:
def query(text):
    corpus_tfidf,index,tfidf = training(docs_list)
    doc_selection = query_of_similarity1(text,index,tfidf)
    doc_pick, value = max(enumerate(doc_selection), key=operator.itemgetter(1))
    
    

0


In [72]:
max(doc_selection[0])

0.053331085

In [73]:
doc_selection[0][0]

0.053331085

In [75]:
print(doc_selection)

[[  5.33310845e-02   8.86751281e-04   4.15503560e-03   1.66756555e-03
    4.40651085e-04   1.97823439e-03   2.64961692e-03   6.32168434e-04
    4.21013770e-04   9.86735802e-04   2.67169694e-03   7.39816669e-03
    2.01833504e-03   2.78602005e-03   2.63986597e-03   1.87958218e-03
    5.31188259e-03   3.69689008e-03   6.81155780e-03   6.71268348e-03
    1.00569078e-03   1.61126512e-03   2.50665750e-03   2.21227109e-03
    8.30254797e-03   1.09263940e-03   2.18016561e-03   1.68163609e-03
    3.25935846e-03   3.57119855e-03   1.27996353e-03   3.39655229e-03
    2.17051757e-03   6.45153574e-04   3.78825190e-03   9.13948752e-04
    2.61177309e-03   1.42839947e-03   2.46087462e-03   1.63897173e-03
    1.48580987e-02   3.59152281e-03   2.26445193e-03   2.97664898e-03
    1.90369133e-03   1.20874122e-03   5.39202360e-04   7.64666125e-03
    6.76457305e-03   3.27414251e-03   9.35599208e-04   2.25979928e-03
    4.24325000e-03   1.44878798e-03   1.19575136e-03   1.19061628e-03
    5.73529862e-03  

In [79]:
np.argsort(doc_selection)

array([[212, 295, 150, 195, 117, 417, 315, 220, 173, 147, 393,  61, 337,
        143, 149,  72, 241,  64, 122, 168, 408,  71, 203, 240, 382,  60,
        292, 333, 405, 219, 389, 165, 204, 141, 347, 131,  78, 246,   8,
        138, 429, 366, 371,   4, 387, 137, 364, 108, 229, 266, 425, 127,
        249,  46,  79, 293, 310, 388, 214, 119,   7, 392, 177,  33, 440,
        282, 332, 129, 253, 189, 207, 438, 182, 268, 346,  66, 436, 111,
        410,   1, 360, 171, 342,  92, 316,  35, 289, 259,  69,  50, 301,
        260,   9, 270,  75, 378,  20, 261, 419, 100, 280, 175, 422,  25,
        340, 341,  87, 109, 251,  96,  76, 390, 254, 383, 170, 312,  55,
         54, 238,  45, 158, 228,  68, 104,  67,  99, 116, 162, 164, 385,
        128, 120, 112,  30, 330, 434, 309, 372, 352, 194, 201, 110, 193,
        331, 255,  37, 132, 344, 151,  86, 325, 118,  53, 354, 365, 328,
        152, 236, 242, 179, 258, 230, 206, 415, 409, 319,  21, 379, 284,
        191, 399, 373,  39, 406, 263,   3,  27,  98

In [80]:
import scipy.stats as ss
ss.rankdata(doc_selection[0])

array([ 442.,   80.,  338.,  176.,   44.,  203.,  264.,   61.,   39.,
         93.,  267.,  405.,  210.,  272.,  263.,  196.,  371.,  316.,
        397.,  393.,   97.,  167.,  256.,  230.,  417.,  104.,  228.,
        177.,  300.,  312.,  134.,  306.,  225.,   64.,  322.,   86.,
        262.,  146.,  251.,  173.,  439.,  315.,  235.,  283.,  198.,
        120.,   54.,  407.,  394.,  303.,   90.,  234.,  341.,  153.,
        118.,  117.,  380.,  241.,  220.,  244.,   26.,   12.,  217.,
        434.,   18.,  376.,   76.,  125.,  123.,   89.,  305.,   22.,
         16.,  236.,  179.,   95.,  111.,  339.,   37.,   55.,  409.,
        271.,  204.,  231.,  213.,  243.,  150.,  107.,  416.,  201.,
        292.,  395.,   84.,  214.,  378.,  223.,  110.,  382.,  178.,
        126.,  100.,  246.,  427.,  324.,  124.,  181.,  281.,  323.,
         48.,  108.,  142.,   78.,  133.,  265.,  191.,  415.,  127.,
          5.,  152.,   60.,  132.,  195.,   19.,  226.,  287.,  189.,
        349.,   52.,